In [1]:
import numpy as np

In [4]:
arr = np.array([
    [1, 2, 3],
    [3, 4, 5],
    [5, 6, 7]
])

In [5]:
np.roll(arr, 1, axis=1)

array([[3, 1, 2],
       [5, 3, 4],
       [7, 5, 6]])

In [6]:
np.roll(arr, -1, axis=1)

array([[2, 3, 1],
       [4, 5, 3],
       [6, 7, 5]])

In [7]:
t = np.roll(arr, 1, axis=1)
np.roll(t, -1, axis=1)

array([[1, 2, 3],
       [3, 4, 5],
       [5, 6, 7]])

In [8]:
t = np.roll(arr, 2, axis=1)
np.roll(t, -2, axis=1)

array([[1, 2, 3],
       [3, 4, 5],
       [5, 6, 7]])

In [1]:
class Action:
    
    def __init__(self, func):
        self.function = func
        
    def run(self, state, params):
        return self.function(state, params)

In [31]:
class Scenario:
    
    def __init__(self, actions):
        self.actions = actions
        
    def run(self, state, params):
        for action in self.actions:
            state = action.run(state, params)
        
        return state

In [32]:
def interpolate_int(start, end, steps):
    return list(np.linspace(start, end, steps+2).astype(int))

def interpolate_float(start, end, steps):
    return list(np.linspace(start, end, steps+2))

def interpolate_str(start, end, steps):
    first = steps // 2
    return [start,] * (first + 1) + [end,] * (steps - first + 1)

def interpolate_vec(start, end, steps):
    raise Exception('Not implemented error')

In [33]:
def test_interp():
    s = 1
    e = 10
    i = interpolate_int(s, e, 8)
    print(i)
    assert i == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    s = 1
    e = 2
    i = interpolate_float(s, e, 1)
    print(i)
    assert i == [1, 1.5, 2]
    
    s = 'first'
    e = 'second'
    i = interpolate_str(s, e, 3)
    print(i)
    assert i == ['first',] * 2 + ['second',] * 3

In [34]:
test_interp()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1.0, 1.5, 2.0]
['first', 'first', 'second', 'second', 'second']


In [56]:
class ScenarioRunner:
    
    def __init__(self, scenario):
        self.scenario = scenario
        
    def run(self, init_params, end_params, init_state, steps):
        params_generator = Interpolator.interp(init_params, end_params, steps)
        
        state = init_state.copy()
        for params in params_generator:
            print(f'{state=}')
            print(f'{params=}')
            state = self.scenario.run(state, params)
        
        return state

In [57]:
class Interpolator:
    
    def interp(start, end, steps):
        yield start
        
        all_states_dict = {}
        
        for k in start.keys():
            vs = start[k]
            ve = end[k]
            assert type(vs) == type(ve), 'Type of start and end values must be same'
            
            if vs == ve:
                interps = [vs,] * (steps + 2) 
            # TODO: test with different int types
            if type(vs) is int:
                interps = interpolate_int(vs, ve, steps)
            elif type(vs) is float:
                interps = interpolate_float(vs, ve, steps)
            elif type(vs) is str:
                interps = interpolate_str(vs, ve, steps)
            else:
                raise Exception('Not implemented error')

            all_states_dict[k] = interps
        
        print('All different states:')
        print(all_states_dict)
        
        state = start.copy()
        for i in range(steps):
            for k in state.keys():
                state[k] = all_states_dict[k][i+1]
            
            yield state
        
        yield end

In [60]:
def func_1(state: dict, params: dict):
    state['x'] = state['result']
    state['y'] = state['x'] * params['a']
    return state

def func_2(state: dict, params: dict):
    state['result'] = state['x'] + state['y'] + params['b']
    return state

In [61]:
action1 = Action(func_1)
action2 = Action(func_2)

init_params = {
    'a': 5,
    'b': 10,
}

end_params = {
    'a': 10,
    'b': 1,
}

init_state = {
    'x': 0,
    'y': 0,
    'result': 0
}

steps = 3

scenario = Scenario([action1, action2])

scenario_runner = ScenarioRunner(scenario)

result_state = scenario_runner.run(init_params, end_params, init_state, steps)

state={'x': 0, 'y': 0, 'result': 0}
params={'a': 5, 'b': 10}
All different states:
{'a': [5, 6, 7, 8, 10], 'b': [10, 7, 5, 3, 1]}
state={'x': 0, 'y': 0, 'result': 10}
params={'a': 6, 'b': 7}
state={'x': 10, 'y': 60, 'result': 77}
params={'a': 7, 'b': 5}
state={'x': 77, 'y': 539, 'result': 621}
params={'a': 8, 'b': 3}
state={'x': 621, 'y': 4968, 'result': 5592}
params={'a': 10, 'b': 1}


In [62]:
result_state

{'x': 5592, 'y': 55920, 'result': 61513}

In [63]:
!git clone https://github.com/assafshocher/ResizeRight.git

Cloning into 'ResizeRight'...


In [1]:
import numpy as np

In [2]:
points = np.random.rand(100, 3)
threshold = 0.5

In [7]:
percentile = np.percentile(points[:, 2], threshold * 100)

indices = points[:, 2] >= percentile

In [8]:
points[indices].shape

(50, 3)

In [11]:
def foo(inputs):
    print(f'Inputs: {inputs}')
    print('Func was called')
    
    return 5

In [79]:
from typing import List, Callable, Dict
from queue import Queue

# TODO change list of inputs values to dictionary
# Возможно, что этот огород не нужен и может быть реализован через EventPropagation, но в этом не уверен
class Node:
    _all_nodes = []
    
    def __init__(self, func: Callable[[Dict], None], name='node', outputs=[], inputs=[]):
        self.name = name
        self.outputs: List[Node] = outputs
        self.inputs: List[Node] = inputs
        self.func = func
        self.res = None
        
        # For debug
        Node._all_nodes.append(self)
        
    def add_child(self, func, name='node', inputs=[]):
        node = Node(func, name=name, outputs=[], inputs=[self,]+list(inputs))
        self.outputs.append(node)
        for n in inputs:
            n.outputs.append(node)
        return node
        
    def run(self):
        self.res = self.func([node.res for node in self.inputs])
        
    @staticmethod
    def add_link(node1, node2):
        assert node1 in Node._all_nodes and node2 in Node._all_nodes, 'Nodes consistency error'
        
    @staticmethod
    def remove(node):
        outputs = node.outputs
        inputs = node.inputs
        
        # Remove all connections
        for o in outputs:
            o.inputs.remove(o)
            
        for i in inputs:
            i.outputs.remove(i)
            
        Node._all_nodes.remove(node)

class Tree:
    
    def __init__(self, root):
        self.root = root
    
    def clear_tree(self):
        pass
    
    def calculate(self):
        stack = list()
        
        self.root.run()
        
        stack += self.root.outputs
        
        while len(stack) > 0:
            node = stack.pop()
            
            is_none = [n for n in node.inputs if n.res is None]
            if len(is_none) > 0:
                stack.append(node)
                
                for n in is_none:
                    stack.append(n)
            else:
                if node.res is None:
                    node.run()
                    for n in node.outputs:
                        stack.append(n)
    def add_node(self):
        pass
    
    def print_tree(self):
        print(root.name)
        n = root
        
        cur_level = n.outputs
        next_level = []
        
        # Iterate every deep level
        while True:
            s1 = ''
            s2 = ''
            for child in cur_level:
                s1 += '|\t'
                s2 += child.name + '\t'
                
                next_level += child.outputs
            print(s1)
            print(s2)
            if len(next_level) == 0:
                break
                
            cur_level = set(next_level)
            next_level.clear()

In [80]:
root = Node(foo, name='root')

c1 = root.add_child(foo, name='c1')
c2 = root.add_child(foo, name='c2')

c3 = c1.add_child(foo, name='c3', inputs=(c2,))
c4 = c2.add_child(foo, name='c4')

tree = Tree(root)

In [81]:
for n in Node._all_nodes:
    print(n.__dict__)

{'name': 'root', 'outputs': [<__main__.Node object at 0x000001F7EC14DAF0>, <__main__.Node object at 0x000001F7EC14D700>], 'inputs': [], 'func': <function foo at 0x000001F7EABD9A60>, 'res': None}
{'name': 'c1', 'outputs': [<__main__.Node object at 0x000001F7EC14D820>], 'inputs': [<__main__.Node object at 0x000001F7EC14D0D0>], 'func': <function foo at 0x000001F7EABD9A60>, 'res': None}
{'name': 'c2', 'outputs': [<__main__.Node object at 0x000001F7EC14D820>, <__main__.Node object at 0x000001F7EC14DAC0>], 'inputs': [<__main__.Node object at 0x000001F7EC14D0D0>], 'func': <function foo at 0x000001F7EABD9A60>, 'res': None}
{'name': 'c3', 'outputs': [], 'inputs': [<__main__.Node object at 0x000001F7EC14DAF0>, <__main__.Node object at 0x000001F7EC14D700>], 'func': <function foo at 0x000001F7EABD9A60>, 'res': None}
{'name': 'c4', 'outputs': [], 'inputs': [<__main__.Node object at 0x000001F7EC14D700>], 'func': <function foo at 0x000001F7EABD9A60>, 'res': None}


In [82]:
tree.print_tree()

root
|	|	
c1	c2	
|	|	
c3	c4	


In [50]:
tree.calculate()

Inputs: []
Func was called
Inputs: [5]
Func was called
Inputs: [5]
Func was called
Inputs: [5]
Func was called
Inputs: [5, 5]
Func was called


In [51]:
for n in Node._all_nodes:
    print(n.__dict__)

{'outputs': [<__main__.Node object at 0x000001F7EABE35E0>, <__main__.Node object at 0x000001F7EABE3C10>], 'inputs': [], 'func': <function foo at 0x000001F7EABD9A60>, 'res': 5}
{'outputs': [<__main__.Node object at 0x000001F7EABE3130>], 'inputs': [<__main__.Node object at 0x000001F7EABE3FD0>], 'func': <function foo at 0x000001F7EABD9A60>, 'res': 5}
{'outputs': [<__main__.Node object at 0x000001F7EABE3130>, <__main__.Node object at 0x000001F7EABE3160>], 'inputs': [<__main__.Node object at 0x000001F7EABE3FD0>], 'func': <function foo at 0x000001F7EABD9A60>, 'res': 5}
{'outputs': [], 'inputs': [<__main__.Node object at 0x000001F7EABE35E0>, <__main__.Node object at 0x000001F7EABE3C10>], 'func': <function foo at 0x000001F7EABD9A60>, 'res': 5}
{'outputs': [], 'inputs': [<__main__.Node object at 0x000001F7EABE3C10>], 'func': <function foo at 0x000001F7EABD9A60>, 'res': 5}


In [52]:
import huggingface_hub

In [53]:
!dir

 ’®¬ ў гбва®©бвўҐ E Ё¬ҐҐв ¬ҐвЄг New SSD
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 1C8F-4F04

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ E:\GitHub\stable_points\notebooks

27.03.2023  19:51    <DIR>          .
27.03.2023  19:51    <DIR>          ..
22.03.2023  19:32    <DIR>          .ipynb_checkpoints
27.03.2023  19:51            16я467 ScenarioTest.ipynb
               1 д ©«®ў         16я467 Ў ©в
               3 Ї Ї®Є  41я460я760я576 Ў ©в бў®Ў®¤­®


In [55]:
huggingface_hub.snapshot_download(repo_id='decapoda-research/llama-7b-hf', cache_dir='J:/Weights/llama-7b-hf')

Fetching 42 files:   0%|          | 0/42 [00:00<?, ?it/s]

c:\users\yamih\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in J:\Weights\llama-7b-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


'J:/Weights/llama-7b-hf\\models--decapoda-research--llama-7b-hf\\snapshots\\5f98eefcc80e437ef68d457ad7bf167c2c6a1348'

In [56]:
huggingface_hub.snapshot_download(repo_id='tloen/alpaca-lora-7b', cache_dir='J:/Weights/alpaca-lora-7b')

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

c:\users\yamih\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in J:\Weights\alpaca-lora-7b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OSError: [WinError 1314] Клиент не обладает требуемыми правами: '..\\..\\blobs\\c7d9f3332a950355d5a77d85000f05e6f45435ea' -> 'J:/Weights/alpaca-lora-7b\\models--tloen--alpaca-lora-7b\\snapshots\\4e4afc5a221b1ce66a5e7bef312e7578aae9b0b1\\.gitattributes'

In [58]:
huggingface_hub.snapshot_download(
    repo_id='decapoda-research/llama-13b-hf',
    cache_dir='J:/Weights/llama-13b-hf',
    ignore_patterns=['.gitattributes'])

Fetching 50 files:   0%|          | 0/50 [00:00<?, ?it/s]

'J:/Weights/llama-13b-hf\\models--decapoda-research--llama-13b-hf\\snapshots\\438770a656712a5072229b62256521845d4de5ce'

In [60]:
huggingface_hub.snapshot_download(
    repo_id='chansung/alpaca-lora-13b',
    cache_dir='J:/Weights/alpaca-lora-13b',
    ignore_patterns=['.gitattributes', 'README.MD'])

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

'J:/Weights/alpaca-lora-13b\\models--chansung--alpaca-lora-13b\\snapshots\\abcdddb2778cace16f184dc1dda0ecf21ade23bc'